In [1]:
import numpy as np

In [2]:
def read_file(path):
    f = open(path, "r")
    func = f.readline().replace('\n', '')
    limitations = []
    f.readline()
    line = f.readline().replace('\n', '')
    while line != '':
        limitations.append(line)
        line = f.readline().replace('\n', '')

    cnt = int(f.readline().replace('\n', ''))
    f.close()
    return (func, limitations, cnt)

In [37]:
class Equation():
    def __init__(self, coefs, sign, bias):
        self.coefs = coefs
        self.sign = sign
        self.bias = bias

    @staticmethod
    def parse(eq, res_sz):
        sign = None
        sp = eq.split('=')
        b = 0
        if '>=' in eq:
            sign = '>='
            sp = eq.split(sign)
            b = eval(sp[1])
            eq = sp[0]
        elif '<=' in eq:
            sign = '<='
            sp = eq.split(sign)
            b = eval(sp[1])
            eq = sp[0]
        elif '=' in eq:
            sign = '='
            sp = eq.split(sign)
            b = eval(sp[1])
            eq = sp[0]

        eq = eq.replace('-', '+-').replace('*', '')
        eq = [x for x in eq.split('+') if x]

        res = [0] * res_sz
        for sub_eq in eq:
            sub_eq = sub_eq.split('x')
            if len(sub_eq) == 1:
                b -= eval(sub_eq[0])
            else:
                pos = eval(sub_eq[1]) - 1
                if (sub_eq[0] == '-'):
                    val = -1
                elif not sub_eq[0]:
                    val = 1
                else:
                    val = eval(sub_eq[0])

                res[pos] += val
        return Equation(res, sign, b)

    def __str__(self):
        return f'{self.coefs} {self.sign} {self.bias}'

In [13]:
def to_canonical(fx, limits, n = None):
    if n is None:
        n = max([len(x.coefs) for x in limits])
    addition = 0
    for eq in limits:
        if len(eq.coefs) < n + addition:
            eq.coefs.extend([0] * (n + addition - len(eq.coefs)))
        if eq.sign == '<=':
            eq.coefs.append(1)
            addition += 1
            eq.sign = '='
        elif eq.sign == '>=':
            eq.coefs.append(-1)
            addition += 1
            eq.sign = '='

    for eq in limits:
        if len(eq.coefs) < n + addition:
            eq.coefs.extend([0] * (n + addition - len(eq.coefs)))
        if eq.bias < 0:
            eq.bias *= -1
            eq.coefs = list(map(lambda x: x * -1, eq.coefs))
    fx.coefs.extend([0] * (n + addition - len(fx.coefs)))
    return fx, limits

In [14]:
def generate_table(canonical_func, canonical_limits):
    limits_matrix = [[x.bias] + x.coefs for x in canonical_limits]
    func = [-canonical_func.bias] + canonical_func.coefs
    limits_matrix = np.array(limits_matrix, dtype=float)
    func = np.array(func, dtype=float)
    diag = np.zeros((limits_matrix.shape[0], limits_matrix.shape[0]), dtype=float)
    np.fill_diagonal(diag, 1)
    simplex_table = np.hstack((limits_matrix, diag))
    penalty = np.sum(limits_matrix, axis=0)
    func = np.hstack((func, [0] * limits_matrix.shape[0]))
    penalty = np.hstack((penalty, [0] * limits_matrix.shape[0]))
    simplex_table = np.vstack((simplex_table, -func, penalty))
    return simplex_table

In [6]:
def find_col(simplex_table, *, debug=False):
    col = 1
    for i in range (2, simplex_table.shape[1]):
        if simplex_table[-1, i] > simplex_table[-1, col] or (simplex_table[-1, i] == simplex_table[-1, col] and simplex_table[-2, i] > simplex_table[-2, col]):
            col = i
    return col

In [7]:
def check_stop(simplex_table):
    for i in range(1, simplex_table.shape[1]):
        if simplex_table[-1, i] > 0:
            return True
        if simplex_table[-2, i] > 0  and abs(simplex_table[-1, i]) == 0:
            return True
    return False

In [8]:
def iterate(simplex_table, base, *, eps=1e-9, debug=False):
    simplex_table_old = np.copy(simplex_table)
    column = find_col(simplex_table, debug=debug)

    with np.errstate(divide='ignore'):
        d = simplex_table[:-2, 0] / simplex_table[: -2, column]
    d[simplex_table[:-2, column] <= 0] = np.NAN

    try:
        row = np.nanargmin(d)
    except ValueError:
        raise RuntimeError('None or infinity solutions')

    base[row] = column

    if debug:
        with np.printoptions(precision=3, suppress=True):
            print(f'd: {d}')
            print(f'column: {column}, row: {row}, a_rl: {simplex_table[row, column]}')
            print(simplex_table)

    simplex_table[row, :] /= simplex_table_old[row, column]
    simplex_table[:, column] = 0
    simplex_table[row, column] = 1

    for i in range(simplex_table.shape[0]):
        for j in range(simplex_table.shape[1]):
            if i == row or j == column:
                continue
            simplex_table[i, j] = simplex_table_old[i, j] - (simplex_table_old[row, j] * simplex_table_old[i, column]) / simplex_table_old[row, column]

    simplex_table[abs(simplex_table) < eps] = 0
    return simplex_table, base

In [58]:
def solve(canonical_func, canonical_limits, *, debug=False):
    simplex_table = generate_table(canonical_func, canonical_limits)
    base_n = simplex_table.shape[0] - 2
    base = list(range(simplex_table.shape[1] - base_n, simplex_table.shape[1]))

    while(check_stop(simplex_table)):
        simplex_table, base = iterate(simplex_table, base, debug=debug)

    res = np.zeros(simplex_table.shape[1] - base_n)
    if max(base) >= len(res):
         raise RuntimeError('None solutions')
    res[base] = simplex_table[range(base_n), 0]
    res[0] = simplex_table[-2, 0]
    
    return res

In [49]:
from os import path
def test(file, base_path):
    func, limits, cnt = read_file(path.join(base_path, file))
    func = Equation.parse(func, cnt)
    limits = list(map(lambda x: Equation.parse(x, cnt), limits))
    canonical_func, canonical_limits = to_canonical(func, limits, cnt)
    with np.printoptions(precision=3, suppress=True):
        print(solve(canonical_func, canonical_limits, debug=True))

In [50]:
test_files = ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt']
base_path = r'C:\Users\mgsds\Projects\Primat-5th-Semester\lab 1\tests'
for f in test_files:
    test(f, base_path)

d: [1.333 0.8  ]
column: 1, row: 1, a_rl: 5.0
[[ 4.  3.  1. -1.  1.  1.  0.]
 [ 4.  5.  1.  1. -1.  0.  1.]
 [-0.  6.  1.  4. -5. -0. -0.]
 [ 8.  8.  2.  0.  0.  0.  0.]]
d: [ 1. nan]
column: 4, row: 0, a_rl: 1.6
[[ 1.6  0.   0.4 -1.6  1.6  1.  -0.6]
 [ 0.8  1.   0.2  0.2 -0.2  0.   0.2]
 [-4.8  0.  -0.2  2.8 -3.8  0.  -1.2]
 [ 1.6  0.   0.4 -1.6  1.6  0.  -1.6]]
d: [4. 4.]
column: 2, row: 0, a_rl: 0.25
[[ 1.     0.     0.25  -1.     1.     0.625 -0.375]
 [ 1.     1.     0.25   0.     0.     0.125  0.125]
 [-1.     0.     0.75  -1.     0.     2.375 -2.625]
 [ 0.     0.     0.     0.     0.    -1.    -1.   ]]
d: [nan  0.]
column: 3, row: 1, a_rl: 1.0
[[ 4.   0.   1.  -4.   4.   2.5 -1.5]
 [ 0.   1.   0.   1.  -1.  -0.5  0.5]
 [-4.   0.   0.   2.  -3.   0.5 -1.5]
 [ 0.   0.   0.   0.   0.  -1.  -1. ]]
[-4.  0.  4.  0.  0.]
d: [4. 0.]
column: 3, row: 1, a_rl: 1.0
[[ 4. -1.  3.  1.  2.  1.  0.]
 [ 0.  1. -1.  1.  0.  0.  1.]
 [-0.  1.  2.  3. -1. -0. -0.]
 [ 4.  0.  2.  2.  2.  0.  0.]]
d:

In [40]:
test('task.txt', 'C:/Users/mgsds/Projects/Primat-5th-Semester/lab 2/files')

[2316.     230.      68.       0.       0.       0.433   24.       1.
   18.333]


In [57]:
func, limits, cnt = read_file(path.join('C:/Users/mgsds/Projects/Primat-5th-Semester/lab 2/files', 'task.txt'))
func = Equation.parse(func, cnt)
limits = list(map(lambda x: Equation.parse(x, cnt), limits))
canonical_func, canonical_limits = to_canonical(func, limits, cnt)
solve(canonical_func, canonical_limits, debug=False)

array([2.50266667e+03, 1.83333333e+02, 6.80000000e+01, 4.66666667e+01,
       0.00000000e+00, 0.00000000e+00, 2.04102564e+01, 2.07692308e+00,
       1.83333333e+01])